In [1]:
import numpy as np
import pandas as pd

from sklearn.cluster import KMeans
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from lightgbm import LGBMRegressor
from sklearn.model_selection import RandomizedSearchCV

In [2]:
SEED = 28
np.random.seed(SEED)

In [3]:
train_data = pd.read_csv('ttids21/train.csv')

In [4]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         50000 non-null  int64  
 1   engine_capacity    19950 non-null  float64
 2   type               43746 non-null  object 
 3   registration_year  50000 non-null  int64  
 4   gearbox            47959 non-null  object 
 5   power              50000 non-null  int64  
 6   model              47743 non-null  object 
 7   mileage            50000 non-null  int64  
 8   fuel               46417 non-null  object 
 9   brand              50000 non-null  object 
 10  damage             41734 non-null  float64
 11  zipcode            50000 non-null  int64  
 12  insurance_price    42669 non-null  float64
 13  price              50000 non-null  int64  
dtypes: float64(3), int64(6), object(5)
memory usage: 5.3+ MB


In [5]:
train_data.head()

,Unnamed: 0,engine_capacity,type,registration_year,gearbox,power,model,mileage,fuel,brand,damage,zipcode,insurance_price,price
0,48298,2.0,bus,2006,auto,140,c4,150000,gasoline,citroen,0.0,49191,380.0,4267
1,81047,NaN,NaN,2016,NaN,0,vito,150000,NaN,mercedes_benz,NaN,45896,NaN,2457
2,92754,2.2,limousine,2010,manual,175,mondeo,125000,diesel,ford,0.0,59229,930.0,10374
3,46007,NaN,NaN,2000,auto,265,andere,150000,gasoline,ford,0.0,39365,680.0,7098
4,76981,NaN,convertible,3,manual,109,2_reihe,150000,gasoline,peugeot,0.0,55271,NaN,2365


In [6]:
train_data.isnull().sum()

Unnamed: 0               0
engine_capacity      30050
type                  6254
registration_year        0
gearbox               2041
power                    0
model                 2257
mileage                  0
fuel                  3583
brand                    0
damage                8266
zipcode                  0
insurance_price       7331
price                    0
dtype: int64

In [7]:
train_data.type.value_counts()

limousine        13392
small car        10270
station wagon     9423
bus               4329
convertible       3284
coupé             2643
other              405
Name: type, dtype: int64

In [8]:
train_data.type.unique()

array(['bus', nan, 'limousine', 'convertible', 'station wagon',
       'small car', 'coupé', 'other'], dtype=object)

In [9]:
train_data.registration_year.unique()

array([2006, 2016, 2010, 2000,    3, 1999, 2005, 2009, 2008, 2001, 2014,
       2002, 1991, 1998, 2003, 1997, 2004, 2012,    0,    9, 2015,   93,
       2013,   96,   99, 2011, 1992, 1996,    4, 1984,   11, 2007,   95,
       1994, 1995,   16,   83,   14, 1983, 1985, 1980, 1966, 1977,   15,
         86, 1993,    1, 1989,   10,   97,   98, 1988, 1967,    2, 1990,
          8,    7, 1981,    6,   12,   85, 1982,    5, 1968, 1976, 1974,
       1969,   74, 1959, 1970,   92, 1972,   90,   72, 1979, 1954,   94,
       1978, 1961, 1973, 1986,   78, 1987, 1975,   91,   82, 1965, 1960,
         13,   89,   88, 1962, 1963, 1964, 1953,   79, 1971, 1958,   77,
         87,   73,   81, 1955,   63,   84,   65, 1951,   54, 1957,   80,
         71,   69, 1956,   70,   60, 1945, 1950, 1952,   75,   68])

In [10]:
# registration_year (-> 4 digit format)

In [11]:
train_data.gearbox.value_counts()

manual    37008
auto      10951
Name: gearbox, dtype: int64

In [12]:
train_data.power.value_counts()

0        4294
75       3162
150      2196
140      1966
60       1953
         ... 
13616       1
650         1
426         1
362         1
415         1
Name: power, Length: 452, dtype: int64

In [13]:
train_data.power.unique()

array([  140,     0,   175,   265,   109,   122,   165,    55,   105,
         116,   115,    86,   120,   231,   220,   136,   118,   160,
         100,    60,   163,   150,   110,    95,   179,   245,   129,
          75,    90,   170,   320,   329,   286,    77,   196,   143,
         180,   209,   101,   167,    54,    97,   125,   131,    58,
         192,    44,   156,   272,   141,    15,    65,    69,    98,
          41,   102,   218,   230,    68,    47,   158,    61,   174,
         200,   193,   177,   106,   240,   239,   124,   224,    71,
          80,   145,   367,   250,   343,   190,    50,    74,   306,
         144,    88,   197,   204,   114,    59,  1362,    70,   345,
         211,   256,   152,   396,   207,    91,   130,   435,   408,
         107,    45,    62,    87,   147,   232,   103,   344,   121,
          82,   340,   300,   205,   203,    64,   155,   113,   184,
         280,   260,   117,  1502,    53,   225,   450,   254,   258,
         241,    84,

In [14]:
train_data.brand.value_counts()

volkswagen        10810
bmw                5696
opel               5087
mercedes_benz      4977
audi               4640
ford               3184
renault            2241
peugeot            1517
fiat               1186
seat                963
skoda               803
mazda               744
smart               712
citroen             696
nissan              689
toyota              626
hyundai             539
mini                529
sonstige_autos      482
volvo               434
mitsubishi          378
kia                 362
honda               360
alfa_romeo          314
porsche             304
suzuki              272
chevrolet           258
chrysler            175
dacia               138
jeep                136
land_rover          118
subaru              102
jaguar              100
daihatsu             99
saab                 71
lancia               66
rover                56
trabant              54
daewoo               54
lada                 28
Name: brand, dtype: int64

In [15]:
train_data.damage.unique()

array([ 0., nan,  1.])

In [16]:
train_data.fuel.value_counts()

gasoline                   29868
diesel                     15665
liquefied petroleum gas      777
compressed natural gas        87
other                         20
Name: fuel, dtype: int64

In [17]:
train_data.head()

,Unnamed: 0,engine_capacity,type,registration_year,gearbox,power,model,mileage,fuel,brand,damage,zipcode,insurance_price,price
0,48298,2.0,bus,2006,auto,140,c4,150000,gasoline,citroen,0.0,49191,380.0,4267
1,81047,NaN,NaN,2016,NaN,0,vito,150000,NaN,mercedes_benz,NaN,45896,NaN,2457
2,92754,2.2,limousine,2010,manual,175,mondeo,125000,diesel,ford,0.0,59229,930.0,10374
3,46007,NaN,NaN,2000,auto,265,andere,150000,gasoline,ford,0.0,39365,680.0,7098
4,76981,NaN,convertible,3,manual,109,2_reihe,150000,gasoline,peugeot,0.0,55271,NaN,2365


In [18]:
train_data.dropna(subset=['engine_capacity'])

,Unnamed: 0,engine_capacity,type,registration_year,gearbox,power,model,mileage,fuel,brand,damage,zipcode,insurance_price,price
0,48298,2.0,bus,2006,auto,140,c4,150000,gasoline,citroen,0.0,49191,380.0,4267
2,92754,2.2,limousine,2010,manual,175,mondeo,125000,diesel,ford,0.0,59229,930.0,10374
8,74568,1.6,NaN,2016,manual,105,golf,150000,gasoline,volkswagen,NaN,79268,30.0,2275
9,20894,1.8,limousine,2005,manual,116,primera,150000,gasoline,nissan,0.0,23554,330.0,4732
10,34481,1.6,limousine,2010,manual,115,astra,125000,gasoline,opel,0.0,75173,460.0,6269
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49987,32954,2.0,station wagon,5,manual,170,4_reihe,150000,diesel,peugeot,0.0,65439,NaN,2502
49988,12417,2.6,bus,2003,NaN,129,sprinter,150000,diesel,mercedes_benz,0.0,34632,920.0,10910
49994,2397,1.6,NaN,2010,manual,105,duster,90000,gasoline,dacia,0.0,61184,290.0,6961
49995,50429,1.4,limousine,2006,manual,75,golf,90000,gasoline,volkswagen,0.0,35745,500.0,4686


In [19]:
X = train_data.drop(columns=['Unnamed: 0', 'price'])
y = train_data.price

In [20]:
train_X, val_X, train_y, val_y = train_test_split(X, y, train_size=0.8, random_state=SEED)

In [21]:
def restore_year(x):
    if x < 50:
        x += 2000
    elif x < 100:
        x += 1900
    return x

In [22]:
train_X.registration_year.map(restore_year).unique()

array([2007, 2004, 1995, 2008, 2011, 2000, 2001, 1996, 1984, 2003, 1998,
       1994, 2006, 2002, 2005, 1993, 1989, 2009, 2016, 1991, 1969, 1999,
       2013, 2010, 2015, 1981, 1970, 1990, 1983, 2014, 2012, 1997, 1982,
       1987, 1985, 1992, 1988, 1980, 1977, 1986, 1964, 1962, 1967, 1979,
       1972, 1978, 1968, 1951, 1966, 1976, 1957, 1974, 1961, 1965, 1963,
       1971, 1954, 1975, 1973, 1960, 1958, 1959, 1955, 1950, 1956, 1945,
       1953, 1952])

In [23]:
train_X.registration_year = train_X.registration_year.map(restore_year)
val_X.registration_year = val_X.registration_year.map(restore_year)

/Users/fliahin/opt/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:5494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [24]:
val_X.registration_year.unique()

array([1986, 2004, 1999, 2013, 2007, 2003, 2009, 2005, 2016, 1998, 2000,
       1995, 2014, 2001, 2006, 2011, 2015, 2010, 2012, 1985, 1996, 2008,
       1991, 1983, 1970, 2002, 1997, 1994, 1992, 1987, 1988, 1977, 1993,
       1990, 1971, 1973, 1980, 1957, 1984, 1989, 1982, 1967, 1972, 1979,
       1969, 1976, 1981, 1964, 1961, 1978, 1955, 1974, 1960, 1963, 1965,
       1968, 1975, 1962, 1956, 1959])

In [25]:
train_X

,engine_capacity,type,registration_year,gearbox,power,model,mileage,fuel,brand,damage,zipcode,insurance_price
40853,NaN,NaN,2007,NaN,0,a_klasse,125000,diesel,mercedes_benz,NaN,45897,NaN
21092,NaN,limousine,2004,manual,75,golf,150000,gasoline,volkswagen,0.0,41238,140.0
37240,NaN,small car,1995,manual,60,golf,150000,gasoline,volkswagen,NaN,51515,NaN
47160,1.6,limousine,2008,manual,179,astra,125000,gasoline,opel,0.0,85640,120.0
13745,1.6,small car,2011,manual,235,corsa,60000,gasoline,opel,0.0,55296,680.0
...,...,...,...,...,...,...,...,...,...,...,...,...
40195,3.0,limousine,2005,auto,224,a6,150000,diesel,audi,0.0,56626,710.0
7200,NaN,small car,2005,manual,60,corsa,150000,gasoline,opel,0.0,91572,120.0
26117,NaN,convertible,2000,auto,0,fortwo,100000,NaN,smart,NaN,13089,80.0
4089,1.4,limousine,2004,manual,102,golf,150000,gasoline,volkswagen,0.0,34593,260.0


In [26]:
si = SimpleImputer(strategy='most_frequent')

In [27]:
imputed_train_X = pd.DataFrame(si.fit_transform(train_X))
imputed_val_X = pd.DataFrame(si.transform(val_X))

imputed_train_X.columns = train_X.columns
imputed_val_X.columns = val_X.columns

In [28]:
imputed_train_X.isnull().sum()

engine_capacity      0
type                 0
registration_year    0
gearbox              0
power                0
model                0
mileage              0
fuel                 0
brand                0
damage               0
zipcode              0
insurance_price      0
dtype: int64

In [29]:
le = LabelEncoder()

In [30]:
imputed_train_X.columns

Index(['engine_capacity', 'type', 'registration_year', 'gearbox', 'power',
       'model', 'mileage', 'fuel', 'brand', 'damage', 'zipcode',
       'insurance_price'],
      dtype='object')

In [31]:
categorical_columns = ['type', 'model', 'fuel', 'brand']

In [32]:
encoders = dict()

In [33]:
for col in categorical_columns:
    le = le.fit(np.append(imputed_train_X[col], ['-1']))
    encoders[col] = le
    imputed_train_X[col] = le.transform(imputed_train_X[col])
    classes = set(val_X[col]) - set(le.classes_)
    imputed_val_X[col].replace(classes, '-1', inplace=True)
    imputed_val_X[col] = le.transform(imputed_val_X[col])

In [34]:
imputed_train_X.replace(['manual', 'auto'], [0, 1], inplace=True)
imputed_val_X.replace(['manual', 'auto'], [0, 1], inplace=True)

In [35]:
imputed_train_X

,engine_capacity,type,registration_year,gearbox,power,model,mileage,fuel,brand,damage,zipcode,insurance_price
0,2.0,4,2007,0,0,34,125000,2,21,0.0,45897,70.0
1,2.0,4,2004,0,75,118,150000,3,39,0.0,41238,140.0
2,2.0,6,1995,0,60,118,150000,3,39,0.0,51515,70.0
3,1.6,4,2008,0,179,44,125000,3,25,0.0,85640,120.0
4,1.6,6,2011,0,235,85,60000,3,25,0.0,55296,680.0
...,...,...,...,...,...,...,...,...,...,...,...,...
39995,3.0,4,2005,1,224,32,150000,2,2,0.0,56626,710.0
39996,2.0,6,2005,0,60,85,150000,3,25,0.0,91572,120.0
39997,2.0,2,2000,1,0,108,100000,3,33,0.0,13089,80.0
39998,1.4,4,2004,0,102,118,150000,3,39,0.0,34593,260.0


In [36]:
test_data = pd.read_csv('ttids21/test_no_target.csv')

In [37]:
test_data

,Unnamed: 0,engine_capacity,type,registration_year,gearbox,power,model,mileage,fuel,brand,damage,zipcode,insurance_price
0,60314,1.6,small car,2013,manual,136,swift,40000,gasoline,suzuki,0.0,30449,490.0
1,12566,NaN,coupé,2004,auto,333,6er,150000,gasoline,bmw,0.0,45307,670.0
2,17760,NaN,station wagon,2006,auto,170,e_klasse,150000,diesel,mercedes_benz,0.0,59494,460.0
3,8876,NaN,limousine,99,manual,101,astra,150000,gasoline,opel,NaN,25524,NaN
4,80392,NaN,limousine,1975,manual,54,andere,150000,diesel,mercedes_benz,0.0,70794,1110.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,93878,1.4,limousine,1999,manual,86,corolla,150000,gasoline,toyota,0.0,44339,110.0
49996,99783,NaN,station wagon,2002,auto,184,3er,150000,diesel,bmw,0.0,47574,240.0
49997,57399,NaN,small car,2005,manual,52,fox,100000,gasoline,volkswagen,0.0,50389,60.0
49998,97106,NaN,bus,2001,manual,151,transporter,150000,diesel,volkswagen,0.0,12209,930.0


In [38]:
test_data.index = test_data['Unnamed: 0']

In [39]:
indexes = test_data.index

In [40]:
test_data.drop(columns=['Unnamed: 0'], inplace=True)

In [41]:
test_data.registration_year = test_data.registration_year.map(restore_year)

In [42]:
imputed_test = pd.DataFrame(si.transform(test_data))
imputed_test.columns = test_data.columns

In [43]:
imputed_test.index = indexes

In [44]:
for col in categorical_columns:
    le = encoders[col]
    unknown_classes = set(imputed_test[col]) - set(le.classes_)
    imputed_test[col].replace(unknown_classes, '-1', inplace=True)
    imputed_test[col] = le.transform(imputed_test[col])

In [45]:
imputed_test.replace(['manual', 'auto'], [0, 1], inplace=True)

In [46]:
imputed_test

,engine_capacity,type,registration_year,gearbox,power,model,mileage,fuel,brand,damage,zipcode,insurance_price
Unnamed: 0,,,,,,,,,,,,
60314,1.6,0,2013,0,136,0,40000,0,36,0.0,30449,490.0
12566,2.0,0,2004,1,333,0,150000,0,3,0.0,45307,670.0
17760,2.0,0,2006,1,170,0,150000,0,21,0.0,59494,460.0
8876,2.0,0,1999,0,101,0,150000,0,25,0.0,25524,70.0
80392,2.0,0,1975,0,54,0,150000,0,21,0.0,70794,1110.0
...,...,...,...,...,...,...,...,...,...,...,...,...
93878,1.4,0,1999,0,86,0,150000,0,37,0.0,44339,110.0
99783,2.0,0,2002,1,184,0,150000,0,3,0.0,47574,240.0
57399,2.0,0,2005,0,52,0,100000,0,39,0.0,50389,60.0


In [47]:
def mean_absolute_percentage_error(y_pred, y_true):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [48]:
params = {
    'boosting_type': ['gbdt', 'dart'],
    'num_leaves': range(1, 100),
    'max_depth': range(2, 100),
    'learning_rate': [0.005, 0.01, 0.03, 0.05, 0.001, 0.0001],
    'reg_lambda': [0, 0.0001, 0.001, 0.01, 0.1]
}

In [49]:
lightgbm_rfc = LGBMRegressor()

In [50]:
def mape(estimator, x, y):
    y_pred = estimator.predict(x)
    y_pred = np.array(y_pred)
    y = np.array(y)
    return mean_absolute_percentage_error(y_pred, y)

In [51]:
rs = RandomizedSearchCV(lightgbm_rfc, params, random_state=SEED, scoring=mape, verbose=1)

In [52]:
rs.fit(imputed_train_X, train_y)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


RandomizedSearchCV(estimator=LGBMRegressor(),
                   param_distributions={'boosting_type': ['gbdt', 'dart'],
                                        'learning_rate': [0.005, 0.01, 0.03,
                                                          0.05, 0.001, 0.0001],
                                        'max_depth': range(2, 100),
                                        'num_leaves': range(1, 100),
                                        'reg_lambda': [0, 0.0001, 0.001, 0.01,
                                                       0.1]},
                   random_state=28, scoring=<function mape at 0x7ff35fa551f0>,
                   verbose=1)

In [53]:
best_reg = rs.best_estimator_

In [54]:
best_reg

LGBMRegressor(learning_rate=0.0001, max_depth=17, num_leaves=5,
              reg_lambda=0.0001)

In [55]:
lgbm_reg = LGBMRegressor(**rs.best_params_, n_estimators=1000)

In [56]:
lgbm_reg.fit(imputed_train_X, train_y)

LGBMRegressor(learning_rate=0.0001, max_depth=17, n_estimators=1000,
              num_leaves=5, reg_lambda=0.0001)

In [57]:
y_pred = lgbm_reg.predict(imputed_val_X)

In [58]:
mean_absolute_percentage_error(y_pred, val_y)

209.51927574732733

In [59]:
from sklearn.ensemble import RandomForestRegressor

In [60]:
rf = RandomForestRegressor(500, verbose=2)

In [61]:
params = {
    'boosting_type': ['gbdt', 'dart'],
    'num_leaves': range(1, 100),
    'max_depth': range(2, 100),
    'learning_rate': [0.005, 0.01, 0.03, 0.05, 0.001, 0.0001],
    'reg_lambda': [0, 0.0001, 0.001, 0.01, 0.1]
}

In [62]:
rf.fit(imputed_train_X, train_y)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s


building tree 1 of 500
building tree 2 of 500
building tree 3 of 500
building tree 4 of 500
building tree 5 of 500
building tree 6 of 500
building tree 7 of 500
building tree 8 of 500
building tree 9 of 500
building tree 10 of 500
building tree 11 of 500
building tree 12 of 500
building tree 13 of 500
building tree 14 of 500
building tree 15 of 500
building tree 16 of 500
building tree 17 of 500
building tree 18 of 500
building tree 19 of 500
building tree 20 of 500
building tree 21 of 500
building tree 22 of 500
building tree 23 of 500
building tree 24 of 500
building tree 25 of 500
building tree 26 of 500
building tree 27 of 500
building tree 28 of 500
building tree 29 of 500
building tree 30 of 500
building tree 31 of 500
building tree 32 of 500
building tree 33 of 500
building tree 34 of 500
building tree 35 of 500
building tree 36 of 500
building tree 37 of 500
building tree 38 of 500
building tree 39 of 500
building tree 40 of 500
building tree 41 of 500
building tree 42 of 500
b

[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:  1.4min finished


RandomForestRegressor(n_estimators=500, verbose=2)

In [63]:
y_pred = lgbm_reg.predict(imputed_val_X)

In [64]:
mean_absolute_percentage_error(y_pred, val_y)

209.51927574732733

In [65]:
lgbm = LGBMRegressor(n_estimators=10000)
lgbm.fit(imputed_train_X, train_y)
y_pred = lgbm.predict(imputed_val_X)
mean_absolute_percentage_error(y_pred, val_y)

31.890344406573956

In [66]:
y_pred = lgbm.predict(imputed_test)

In [67]:
y_pred = pd.DataFrame(data={
    'id': indexes, 
    'Predicted': y_pred}, index=indexes)

y_pred.to_csv('submission.csv',
        columns=['id', 'Predicted'], index=False)